# Transformación de datos

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# Carga de datos
data_frame = pd.read_csv("Churn_Modelling.csv")
data_frame.head()

In [ ]:
BYTES_TO_MB_DIV = 0.000001
def print_memory_usage_of_data_frame(df):
    mem = round(df.memory_usage().sum() * BYTES_TO_MB_DIV, 3) 
    print("Memory usage is " + str(mem) + " MB")
print_memory_usage_of_data_frame(data_frame)

In [ ]:
data_frame.shape

In [ ]:
# Valores nulos
for feature in data_frame.columns:
    print('Total de valores nulos de', feature, '=', data_frame[feature].isna().sum())

In [ ]:
# Selecciona solo las características que se van a considerar en los posteriores análisis
X = data_frame.iloc[:, 3:-1].values
# La ùltima columna (sale o no sale) se separa del resto para analizar la relación que tiene ella con el resto
y = data_frame.iloc[:, -1].values

In [ ]:
pd.DataFrame(X).head(10)

In [ ]:
pd.DataFrame(y).head(5)

## Transformación de variables categóricas

Una **variable categórica** es aquella que toma valores desde un conjunto limitado de elementos. 

Se revisarán 3 enfoques para tratar este tipo de variables. Antes de revisarlos es conveniente preguntar ¿por qué deben ser tratadas?, la respuesta: los algoritmos que se revisarán más adelante requieren, muchos de ellos, que las variables sean numéricas.

Los enfoques son:

1. Borrar las variables categóricas. Se aplica cuando la columna no aporta mayor valor al análisis.
2. Etiqueta codificada (Label Encoding(. Asigna a cada valor de la lista un número entero diferente. Se debe tener cuidado porque el orden de los números no necesariamente representa el orden de las categorías.
3. One-hot-encoding. Crea nuevas columnas indicando la presencia (o ausencia) de cada posible valor en el set de datos original.

Antes de comenzar a trabajar con nuestro set de datos, se revisará primero la forma en que trabaja el Label y el one-hot encoding

In [ ]:
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder

# define example
data = ['frio', 'frio', 'templado', 'frio', 'calor', 'calor', 'templado', 'frio', 'templado', 'calor']
values = array(data)
print('Valores:',values)
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
print('Label encoder integer:',integer_encoded)
print('Label encoder Clases:',label_encoder.classes_)

In [ ]:
type(integer_encoded)

In [ ]:
# binary encode
from sklearn.preprocessing import OneHotEncoder
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print('Categorias de one-hot encoder:\n', onehot_encoder.categories_)
print('one-hot encoder:\n', onehot_encoded)

# invert first example
inverted = label_encoder.inverse_transform([argmax(onehot_encoded[0, :])])
print(inverted)

#### Analizando el resultado anterior

La matriz que aparece junto one-hot-encoder es de 3x3; esto es porque se tienen tres valores distintos.

Si se revisa la primera fila: [0 1 0] significa que el primer valor corresponde a frío; es decir, la primera columna representa el valor calor (codificado en 0), la segunda es frio (codificada en 1) y la tercera columna es templado (codificada en 2).

### Se comienza con la columna que contiene el género

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])
pd.DataFrame(X)

Para el caso de la columna de Geografía (columna 1 en el dataframe anterior) se usará el tercer enfoque

In [ ]:
# Primero se revisan los valores únicos
pd.DataFrame(X)[1].unique()

Se observan 3 valores distintos, por lo que se espera que se agreguen 3 columnas y que cada una de ellas contenga 0 (ausencia) o 1 (presencia) del valor original

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))
pd.DataFrame(X)

### Comprobando la salida

Se visualizan 3 columnas al inicio de la matriz; si se considera la primera fila es (1,0,0) y sabiendo que el orden sería (France, Germany y Spain) indica que la primera observación sería de **France**.

El mismo análisis se hace para la segunda fila (0,0,1) que corresponde a **Spain**

## Escalar las características

Existen varias alternativas:

1. StandardScaler
2. MinMaxScaler

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

scaled_data = sc.fit_transform(X)
pd.DataFrame(scaled_data).head()

In [ ]:
scaler = MinMaxScaler()
scaled_data_1 = scaler.fit_transform(X)
pd.DataFrame(scaled_data_1).head()

In [ ]:
pd.DataFrame(scaled_data_1).describe()

In [ ]:
df_x = pd.DataFrame(scaled_data)
df_x.describe()